In [ ]:
!pip install sagemaker --upgrade

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.serverless import ServerlessInferenceConfig

# Hub Model configuration. <https://huggingface.co/models>
hub = {
    'HF_MODEL_ID':'distilbert-base-uncased-finetuned-sst-2-english',
    'HF_TASK':'text-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,                      # configuration for loading model from Hub
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version="4.26",  # transformers version used
   pytorch_version="1.13",        # pytorch version used
   py_version='py39',            # python version used
)


In [12]:
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=3072, max_concurrency=10,
)

In [ ]:
predictor = huggingface_model.deploy(
    serverless_inference_config=serverless_config
)

In [ ]:
data = {
  "inputs": "sun is yellow",
}

res = predictor.predict(data=data)
print(res)

In [ ]:
import json

data = {'inputs': 'fuck you'}
body_bytes = json.dumps(data)

runtime = boto3.client(service_name='sagemaker-runtime')
response = runtime.invoke_endpoint(EndpointName='huggingface-pytorch-inference-2023-12-07-13-10-03-171', 
                                   ContentType='application/json', 
                                   Body=body_bytes)
print(json.loads(response['Body'].read()))